In [12]:
import os
basedir = 'C:\\projects\\New_York_OSM\\samples'
names = os.listdir(basedir)
paths = [os.path.join(basedir, name) for name in names[:5]]
sizes = [os.stat(path).st_size for path in paths]
for i in range(5):
    print(names[i], ':', sizes[i])

sample_nodes.csv : 34343067
sample_nodes_tags.csv : 960657
sample_ways.csv : 3927976
sample_ways_nodes.csv : 11669712
sample_ways_tags.csv : 9811433
